Load openai and transliterate the lyrics to roman letters (english).

In [2]:
# Prepare jamendo dataset
# Split lyrics to English and Non-English
# read jamendolyrics/JamendoLyrics.csv
import pandas as pd

df = pd.read_csv('jamendolyrics/JamendoLyrics.csv')

In [3]:
import os
import shutil

models = ["baseline", "bdr_baseline", "MTL", "bdr_MTL"]

for index, row in df.iterrows():
    title = row['Filepath'][:-4]
    lang = row['Language']

    # create the directory if it doesn't exist
    os.makedirs(f'data_pipeline/{lang}/lyrics_src', exist_ok=True)
    os.makedirs(f'data_pipeline/{lang}/lyrics', exist_ok=True)
    os.makedirs(f'data_pipeline/{lang}/mp3', exist_ok=True)
    os.makedirs(f'data_pipeline/{lang}/annotations', exist_ok=True)

    # Copy the lyric and mp3 files
    if lang == "English":
        shutil.copy(f'jamendolyrics/lyrics/{title}.words.txt', f'data_pipeline/{lang}/lyrics/{title}.words.txt')
        shutil.copy(f'jamendolyrics/lyrics/{title}.txt', f'data_pipeline/{lang}/lyrics/{title}.txt')
        shutil.copy(f'jamendolyrics/mp3/{title}.mp3', f'data_pipeline/{lang}/mp3/{title}.mp3')
    else:
        shutil.copy(f'jamendolyrics/lyrics/{title}.txt', f'data_pipeline/{lang}/lyrics_src/{title}.txt')
        shutil.copy(f'jamendolyrics/mp3/{title}.mp3', f'data_pipeline/{lang}/mp3/{title}.mp3')

    # Copy the annotations for both lines and words
    shutil.copy(f'jamendolyrics/annotations/lines/{title}.csv', f'data_pipeline/{lang}/annotations/{title}.lines.csv')
    shutil.copy(f'jamendolyrics/annotations/words/{title}.csv', f'data_pipeline/{lang}/annotations/{title}.words.csv')

    # Create another copy under a prediction directory
    os.makedirs(f'predictions/{lang}/annotations', exist_ok=True)
    shutil.copy(f'jamendolyrics/annotations/lines/{title}.csv', f'predictions/{lang}/annotations/{title}.lines.csv')
    shutil.copy(f'jamendolyrics/annotations/words/{title}.csv', f'predictions/{lang}/annotations/{title}.words.csv')

    # Copy the results of the predictions for each model, if they exist
    for model in models:
        os.makedirs(f'predictions/{lang}/{model}', exist_ok=True)
        if os.path.exists(f'data_pipeline/{lang}/predictions_{model}/{title}_align.csv'):
            shutil.copy(f'data_pipeline/{lang}/predictions_{model}/{title}_align.csv', f'predictions/{lang}/{model}/{title}_align.csv')



In [ ]:
from src.transliteration import process_lyrics

# process_lyrics("data_pipeline/Semitic", 'Arabic Hebrew')

failed = {'Spanish': ['Baila_-_Alfonso_Lugo.txt', '10._Disparan_-_criatura.txt', 'Quiero_y_Puedo_-_Nacidos_de_la_Tierra.txt', 'Besando_Sapos_-_Dream_Tabu.txt', 'Guayeteo_-_JhoyKing.txt'],
          'French': ['POIGNEE_DE_MAIN_-_Cabbac.txt', "Mère_nature_-_Law'.txt", 'En_liberté_-_tom.leyak.txt', 'Une_Vie_De_Roi_-_david.txt'],
          "German": ['dich_gehen_zu_sehn._-_Wingenbach_Frank.txt', 'Drei_Nüsse_-_patrouille.txt', 'Der_Baum_-_Dienstag_is_Damensauna_(2).txt', '1_Freak_-_Automatisch_Gekommen.txt', 'Schnodderdodder_-_Abendblau.txt', 'Keine_Lust_-_Jonny_M.txt']}

for lang in ["French"]:
    process_lyrics(f'data_pipeline/{lang}', lang)